# collect_air_data Class Test
This notebook shows how you can use the `collect_air_data` class to manage air sensors


In [1]:
import time
#Analog sensors setup
from redpitaya.overlay.mercury import mercury as FPGA

import sys #Import from the folder above
sys.path.append("..")

import collect_air_data
import bme680

air_data = collect_air_data.collect_air_object("/dev/ttyPS1",bme680.BME680(i2c_addr=0x76, i2c_device=None),
                 FPGA.analog_in(1), #ANALOG_TEMP_pin
                 FPGA.analog_in(2), #voc_pin
                 FPGA.analog_in(3)) #analog_alcohol_pin
#Manage relative values of PMS here
pms_ref_0_3 = air_data.pms_ref_0_3
pms_ref_0_5 = air_data.pms_ref_0_5
pms_ref_1_0 = air_data.pms_ref_1_0

In [2]:
#The pms5003 is a particle sensor
pms5003_data = air_data.pms5003.read()
print(pms5003_data.pm_per_1l_air(0.3),pms5003_data.pm_per_1l_air(0.3)/pms_ref_0_3)
print(pms5003_data.pm_per_1l_air(0.5),pms5003_data.pm_per_1l_air(0.5)/pms_ref_0_5)
print(pms5003_data.pm_per_1l_air(1.0),pms5003_data.pm_per_1l_air(1.0)/pms_ref_1_0)
print(pms5003_data.pm_per_1l_air(2.5))
print(pms5003_data.pm_per_1l_air(5.0))
print(pms5003_data.pm_per_1l_air(10.0))

114 0.4523809523809524
15 0.38461538461538464
15 0.6
2
0
0


In [3]:
air_data.pms5003.read()


PM1.0 ug/m3 (ultrafine particles):                             0
PM2.5 ug/m3 (combustion particles, organic compounds, metals): 1
PM10 ug/m3  (dust, pollen, mould spores):                      1
PM1.0 ug/m3 (atmos env):                                       0
PM2.5 ug/m3 (atmos env):                                       1
PM10 ug/m3 (atmos env):                                        1
>0.3um in 0.1L air:                                            114
>0.5um in 0.1L air:                                            15
>1.0um in 0.1L air:                                            15
>2.5um in 0.1L air:                                            2
>5.0um in 0.1L air:                                            0
>10um in 0.1L air:                                             0

In [4]:
print(air_data.gas_gmxxxb.getGM102B_volts())
print(air_data.gas_gmxxxb.getGM302B_volts())
print(air_data.gas_gmxxxb.getGM502B_volts())
print(air_data.gas_gmxxxb.getGM702B_volts())

1.1741935483870969
0.832258064516129
0.6451612903225806
0.41935483870967744


In [5]:
while True:
    if air_data.bme_680.get_sensor_data():
        output = '{0:.2f} C,{1:.2f} hPa,{2:.2f} %RH'.format(
            air_data.bme_680.data.temperature,
            air_data.bme_680.data.pressure,
            air_data.bme_680.data.humidity)
        #Only print gas data if the heater is stable
        if air_data.bme_680.data.heat_stable:
            print('{0},{1} Ohms'.format(
                output,
                air_data.bme_680.data.gas_resistance))
            break
        else:
            print(output)
    time.sleep(1)

32.68 C,1014.88 hPa,33.31 %RH
32.66 C,1014.88 hPa,33.32 %RH
32.65 C,1014.91 hPa,33.31 %RH
32.68 C,1014.87 hPa,33.30 %RH,75187.63141422218 Ohms


In [6]:
print(air_data.bme_680.get_sensor_data().gas_resistance)
print(air_data.bme_680.get_sensor_data().gas_volts)
print(air_data.bme_680.get_sensor_data().gas_rel)

77041.22215757887
2.4287714472860658
0.7119602284634317


In [7]:
air_data.analog_temp()

26.915057573267916

In [8]:
print(air_data.analog_voc_ratio())
print(air_data.analog_alcohol_ratio())
print(air_data.GM102B_ratio())
print(air_data.GM302B_ratio())
print(air_data.GM502B_ratio())
print(air_data.GM702B_ratio())

0.8254041404552267
0.6695439132739723
0.7148493347604279
0.6491211801474182
0.6357685291383258
0.4440738179500739


In [9]:
#Reset all the reference values to their current value
pms5003_data = air_data.pms5003.read()
pms_ref_0_3 = pms5003_data.pm_per_1l_air(0.3)
pms_ref_0_5 = pms5003_data.pm_per_1l_air(0.5)
pms_ref_1_0 = pms5003_data.pm_per_1l_air(1.0)
air_data.reset_ref_values()

(0.16605623356087176,
 3.6378298589366236,
 1.1774193548387097,
 0.8354838709677419,
 0.6419354838709677,
 0.41935483870967744,
 2.709552544961243)